In [3]:
import numpy as np
import requests
import zipfile 
from io import StringIO
import pandas as pd
import json
import copy

In [8]:
masterpath = 'http://faostat.fao.org/Portals/_Faostat/Downloads/zip_files/'
#masterpath = 'http://food.csaladen.es/db/' #alternative mirror for entire FAO database
localpath=True
if localpath:
    masterpath = 'E:/Skydrive/Documents/GitHub/Food-Energy/db/' #local path for downloaded FAO database, only for developer purposes
indicators=[]
indicators.append('CommodityBalances_Crops_E_All_Data') #0
indicators.append('CommodityBalances_LivestockFish_E_All_Data') #1
indicators.append('Emissions_Agriculture_Burning_crop_residues_E_All_Data') #2
indicators.append('Emissions_Agriculture_Energy_E_All_Data') #3
indicators.append('Emissions_Agriculture_Synthetic_Fertilizers_E_All_Data') #4
indicators.append('Environment_Energy_E_All_Data') #5
indicators.append('FoodBalanceSheets_E_All_Data') #6
indicators.append('FoodSupply_Crops_E_All_Data') #7
indicators.append('FoodSupply_LivestockFish_E_All_Data') #8
indicators.append('Population_E_All_Data') #9
indicators.append('Production_Crops_E_All_Data') #10
indicators.append('Production_CropsProcessed_E_All_Data') #11
indicators.append('Production_Livestock_E_All_Data') #12
indicators.append('Production_LivestockPrimary_E_All_Data') #13
indicators.append('Production_LivestockProcessed_E_All_Data') #14
indicators.append('Resources_Fertilizers_E_All_Data') #15
indicators.append('Resources_FertilizersArchive_E_All_Data') #16

In [10]:
import pandas as pd

# read the file in
pd.read_csv('http://eventdata.parusanalytics.com/cameo.dir/CAMEO.SCALE.txt',sep=':')

,## CAMEO SCALE,VERSION 0.5B1 [07.03.21]
0,"## Philip Schrodt, schrodt@ku.edu",NaN
1,01,[0.0] MAKE PUBLIC STATEMENT
2,010,"[0.0] Make statement, not specified below"
3,011,[-0.1] Decline comment
4,012,[-0.4] Make pessimistic comment
...,...,...
272,202,[-10.0] Engage in mass killings
273,203,[-10.0] Engage in ethnic cleansing
274,204,"[-10.0] Use weapons of mass destruction, not s..."
275,2041,"[-10.0] Use chemical, biological, or radiologi..."


In [13]:
scale.head()

,cameoCode,Description
0,01,[0.0] MAKE PUBLIC STATEMENT
1,010,"[0.0] Make statement, not specified below"
2,011,[-0.1] Decline comment
3,012,[-0.4] Make pessimistic comment
4,013,[0.4] Make optimistic comment


In [14]:
# to keep track of time
import datetime

start = datetime.datetime.now()

# strip the Goldstein data scale to its own column
scale['GoldsteinScale']=scale.Description.apply(
                                            lambda x: pd.Series(x.split(']')[0].strip('[]'
                                             ),
                                            name='GoldsteinScale'))
end = datetime.datetime.now() - start
print('This operation took {0} microsecnds.\n\n'.format(end.microseconds))
scale.head()

This operation took 75118 microsecnds.




,cameoCode,Description,GoldsteinScale
0,01,[0.0] MAKE PUBLIC STATEMENT,0.0
1,010,"[0.0] Make statement, not specified below",0.0
2,011,[-0.1] Decline comment,-0.1
3,012,[-0.4] Make pessimistic comment,-0.4
4,013,[0.4] Make optimistic comment,0.4


In [12]:
scale = pd.read_csv('http://eventdata.parusanalytics.com/cameo.dir/CAMEO.SCALE.txt',
                    sep=':',
                    header=None,
                    dtype={'cameoCode':'str'},
                    names=['cameoCode','Description'],
                   skiprows=2)

In [15]:
start = datetime.datetime.now()

# strip only description text
scale['Description']=scale.Description.apply(
                                        lambda x: pd.Series(x.split(']')[1],
                                        name='cameoCodeDescription'))
end = datetime.datetime.now() - start
print('This operation took {0} microsecnds.\n\n'.format(end.microseconds))
scale.head()

This operation took 70057 microsecnds.




,cameoCode,Description,GoldsteinScale
0,01,MAKE PUBLIC STATEMENT,0.0
1,010,"Make statement, not specified below",0.0
2,011,Decline comment,-0.1
3,012,Make pessimistic comment,-0.4
4,013,Make optimistic comment,0.4


In [16]:
start = datetime.datetime.now()


# strip whitespace from cameoCode column
scale.cameoCode=scale.cameoCode.apply(lambda x: x.strip())

# set the index to the cameoCode for json lookups
scale.set_index(scale.cameoCode.values,inplace=True)

# rename the index to cameoCode
scale.index.rename('cameoCode',inplace=True)

end = datetime.datetime.now() - start
print('All of these operations took {0} microsecnds total.\n\n'.format(end.microseconds))
scale.head()

All of these operations took 2084 microsecnds total.




,cameoCode,Description,GoldsteinScale
cameoCode,,,
01,01,MAKE PUBLIC STATEMENT,0.0
010,010,"Make statement, not specified below",0.0
011,011,Decline comment,-0.1
012,012,Make pessimistic comment,-0.4
013,013,Make optimistic comment,0.4
